In [109]:
import numpy as np
import gzip
from collections import defaultdict
import random
from tqdm import tqdm_notebook as tqdm
import warnings
from math import *
warnings.filterwarnings('ignore')

In [111]:
data = [eval(l.decode('utf-8')) for l in gzip.open('assignment1/train.json.gz', 'r')]

### Task 1

###### In the task 1, I tried Jaccard Similarity and baseline model. 

At the begining, I built a validation set of positive and negative samples to evaluate the model performance.  
  
In the Jaccard Similarity model, I used an item-category dictionary that restores all the categories that each item belongs to and a user-category dictionary that restores all the categories that each user has purchased. Next step is to compare the similarity between the categories that an item that belongs to and all the categories that users purchased so as to get their average similarity scores. Then based on the average similarity scores that rank in the descending order, the first half pairs are the ones that purchased. However, the accuracy I got in the validation set is 0.6103.  
  
In the baseline model, I also built an item-category dictionary that restores all the categories that each item belongs to and a user-purchased category dictionary that restores all the categories that each user has purchased. If the user is not in the user-purchased category dictionary or the item is not in the item-category dictionary, then I assume that in this pair, user wouldn't purchase this item. If the item is in the item-category dictionary，then whether the user is the purchased dictionary determine whether the user would purchase the item or not. In the baseline model, I chose lambda=5 because I tried a set of lambdas ranging from 0.01 to 100 and the accuracy in the validation set is the highest when the lambda equals to 5. And I split 2000 observations into the validation set so as to train the model better. In this way, I got the final result of 0.65 in the validation set.   
  
Therefore, I chose to use the baseline model for the task 1.

In [113]:
data_ = data
np.random.shuffle(data_)
train, val = data_[:100000], [(d['reviewerID'], d['itemID']) for d in data_[100000:]]
userid = list(set([d['reviewerID'] for d in data_]))
itemid = list(set([d['itemID'] for d in data_]))
user_item = list(set([(d['reviewerID'], d['itemID']) for d in data_]))

f_val = []
for i in tqdm(range(100000)):
    user, item = np.random.choice(userid), np.random.choice(itemid)
    while (user, item) in user_item or (user, item) in f_val:
        user, item = np.random.choice(userid), np.random.choice(itemid)
    f_val.append((user, item))

#### Jaccard Similarity

In [2]:
user_cat, item_cat = {}, {}
for i in train:
    user, item, cats = i['reviewerID'], i['itemID'], i['categories'] 
    if item not in item_cat:
        item_cat[item] = set()
    if user not in user_cat:
        user_cat[user] = set()
    for c in cats:
        for l in c:
            item_cat[item].add(l)
            user_cat[user].add(l)

In [108]:
user_cat, item_cat = {}, {}
for i in train:
    user, item, cats = i['reviewerID'], i['itemID'], i['categories'] 
    if item not in item_cat:
        item_cat[item] = set()
    if user not in user_cat:
        user_cat[user] = set()
    for c in cats:
        for l in c:
            item_cat[item].add(l)
            user_cat[user].add(l)


def jaccard_sim(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

result, all_avg = [], []
count = 0
for u,i in val + f_val:
    if i in item_cat and u in user_cat:
        all_sim = []
        sim = jaccard_sim(user_cat[u],item_cat[i])
        all_sim.append(sim)
        avg_sim = np.mean(all_sim)
        count += avg_sim==0
    else:
        avg_sim = 0    
    result.append((u,i,avg_sim))  
    
u, i, avg_sim = zip(*result)  
avg_sim = sorted(list(avg_sim), reverse = True)
median = avg_sim[len(avg_sim)//2]
print(median, count)
print(sum(np.array(avg_sim)==0))
score = []
for u,i,avg_sim in result:
    if avg_sim >= median:
        s = 1
        score.append((u,i,s))
    else:
        s = 0
        score.append((u,i,s))
ur, ir, sr = zip(*score)

accuracy = sum(np.array(sr) == ([1]*len(val) + [0]*len(f_val)))/len(sr)
print(accuracy) 

0.15384615384615385 0
19917
0.6103


In [ ]:
pred = open('assignment1/prediction_Purchase.txt','w')
for l in open('assignment1/pairs_Purchase.txt','r'):
    if l.startswith('reviewerID'):
        pred.write(l)
        continue
    else:
        u,i = l.strip().split('-')
    if i in i_user and u in u_item:
        all_sim = []
        for user in i_user[i]:
            sim = jaccard_sim(u_item[u],u_item[user])
            all_sim.append(sim)
            if sum(all_sim)/len(all_sim) > 0.3:
                pred.write(u + '-' + i + ",1\n")
            else:
                pred.write(u + '-' + i + ",0\n")
pred.close()

#### Baseline Model

In [ ]:
pur, item_cat = {}, {}
for d in train:
    user, item, cat = d['reviewerID'], d['itemID'], d['categories']
    if user not in pur:
        pur[user] = set()
    if item not in item_cat:
        item_cat[item] = set(map(tuple, cat))
    for c in cat:
        pur[user].add(tuple(c))

result = []
for u, i in val + f_val:
    if i not in item_cat or u not in pur:
        result.append(1)
    else:
        s = 0
        for cat in item_cat[i]:
            if cat in pur[u]:
                s += 1
        if s != 0:
            result.append(1)
        else:
            result.append(0)

accuracy = sum(np.array(result) == ([1]*len(val) + [0]*len(f_val)))/len(result)
print(accuracy)

dat = [(d['reviewerID'], d['itemID'], d['categories']) for d in data_]
pur, item_cat = {}, {}
for user, item, cat in dat:
    if user not in pur:
        pur[user] = set()
    if item not in item_cat:
        item_cat[item] = set(map(tuple, cat))
    for c in cat:
        pur[user].add(tuple(c))
        
pred = open('assignment1/prediction_Purchase.txt','w')
for l in open('assignment1/pairs_Purchase.txt','r'):
    if l.startswith('reviewerID'):
        pred.write(l)
        continue
    else:
        u,i = l.strip().split('-')
    if i not in item_cat or u not in pur:
        pred.write(u + '-' + i + ",0\n")
    else:
        s = 0
        for cat in item_cat[i]:
            if cat in pur[u]:
                s += 1
        if s != 0:
            pred.write(u + '-' + i + ",1\n")
        else:
            pred.write(u + '-' + i + ",0\n")
pred.close()

### Task 2

In this task I tried three models. They are baseline model, y~alpha + beta_i + beta_u and the latent-factor model. It turned out that the latent-factor model worked best in this case.  
  
I split 2000 observations into the validation set so as to train the model better. In the model, I initialized the alpha as the average rating, and initialized beta_i, beta_u, gamma_i, gamma_u as 0 or an array of 0. Then I used gradient descent in the model to find the optimized values. If the user is in the beta_u dictionary and the item is in the beta_i dictionary, then the rating of this pair equals to alpha + beta_i[i] + beta_u[u] + sum(gamma_i[i]*gamma_u[u]). If the user is not in the beta_u dictionary, then I assume beta_u and gamma_u equal to 0. It works the same when the item is not in the beta_i dictionary.   
  
Here I also ran the model with different lambda and dimensions and it turns out the lambda = 5 and dimension =6 would result to a relative lower MSE in the validation set. In this way, I got the final MSE of 1.1359 in the validation set.  

In [9]:
data_ = data
np.random.shuffle(data_)
train = [(d['reviewerID'], d['itemID'], d['rating']) for d in data_[:-2000]]
val = [(d['reviewerID'], d['itemID'], d['rating']) for d in data_[-2000:]]
t_user, t_item, t_rating = zip(*train)
v_user, u_item, u_rating = zip(*val)

lr, lam, dim= 0.01, 5, 6
mse_, mse = 30, 29
alpha = np.mean(t_rating)
beta_i, beta_u, gamma_i, gamma_u = {}, {}, {}, {}
i_count, u_count = {}, {}
for u, i, r in train:
    if i in i_count:
        i_count[i] += 1
    else:
        i_count[i] = 1
    if u in u_count:
        u_count[u] += 1
    else:
        u_count[u] = 1
for i in i_count:
    beta_i[i] = 0
    gamma_i[i] = np.zeros(dim)
for u in u_count:
    beta_u[u] = 0
    gamma_u[u] = np.zeros(dim)
while mse_ - mse > 10**(-6):
    mse_ = mse
    d_alpha = 0
    dbeta_i = {i:2*lam*beta_i[i] for i in i_count}
    dbeta_u = {u:2*lam*beta_u[u] for u in u_count}
    dgamma_i = {i:2*lam*gamma_i[i] for i in i_count}
    dgamma_u = {u:2*lam*gamma_u[u] for u in u_count}
    for u, i, r in train:
        error = alpha + beta_i[i] + beta_u[u] + sum(gamma_i[i]*gamma_u[u]) - r
        d_alpha += 2*error
        dbeta_i[i] += 2*error
        dbeta_u[u] += 2*error
        dgamma_i[i] += 2*error*gamma_u[u]
        dgamma_u[u] += 2*error*gamma_i[i]
    alpha = alpha - (d_alpha/len(train))*lr
    beta_i = {i:beta_i[i]-lr*dbeta_i[i]/i_count[i] for i in i_count}
    beta_u = {u:beta_u[u]-lr*dbeta_u[u]/u_count[u] for u in u_count}
    gamma_i = {i:gamma_i[i]-lr*dgamma_i[i]/i_count[i] for i in i_count}
    gamma_u = {u:gamma_u[u]-lr*dgamma_u[u]/u_count[u] for u in u_count}
    err_2 = 0
    beta_i_mean = np.mean(list(beta_i.values()))
    beta_u_mean = np.mean(list(beta_u.values()))
    for u, i, r in val:
        if u in beta_u and i in beta_i:
            err_2 += (alpha + beta_i[i] + beta_u[u] + sum(gamma_i[i]*gamma_u[u]) - r)**2
        elif u not in beta_u and i in beta_i:
            err_2 += (alpha + beta_i[i] + beta_u_mean - r)**2
        elif u in beta_u and i not in beta_i:
            err_2 += (alpha + beta_i_mean + beta_u[u] - r)**2
        else:
            err_2 += (alpha + beta_i_mean + beta_u_mean - r)**2
    mse = err_2/len(val)

pred = open('assignment1/prediction_Rating.txt','w')
for l in open('assignment1/pairs_Rating.txt','r'):
    if l.startswith('reviewerID'):
        pred.write(l)
        continue
    else:
        u,i = l.strip().split('-')
    if u in beta_u and i in beta_i:
        r = alpha + beta_i[i] + beta_u[u] + sum(gamma_i[i]*gamma_u[u])
    elif u not in beta_u and i in beta_i:
        r = alpha + beta_i[i] + beta_u_mean
    elif u in beta_u and i not in beta_i:
        r = alpha + beta_i_mean + beta_u[u]
    else:
        r = alpha + beta_i_mean + beta_u_mean
    pred.write(u + '-' + i + ',{}\n'.format(r))
pred.close()